In [8]:
import os
import collections
import json
import pickle
from tqdm import tqdm
from natsort import natsorted

import numpy as np
import pandas as pd
import torch
from collections import defaultdict
# pd.set_option('display.max_columns', 500)
# pd.options.display.max_rows = 500
# pd.options.display.max_columns = 20
# pd.set_option('display.max_columns', 200)
# pd.set_option('display.max_rows', 100)
# pd.set_option('display.min_rows', 100)
# pd.set_option('display.expand_frame_repr', True)

# import matplotlib.pyplot as plt
# import seaborn as sns

from src.wimbd_ import _load_dataset, WimbdTasks
wt = WimbdTasks()

In [9]:
# logits_pth = "./models/experiment_6_logits/inference/EleutherAI/pythia-14m/TRANSLATION/wmt09-cs-en/0-shot/logits.pt"
# logits_pth = "./models/experiment_6_logits_max/inference/EleutherAI/pythia-14m/TRANSLATION/wmt09-cs-en/0-shot/logits.pt"
logits_pth = "./models/experiment_6_logits_max_2/inference/EleutherAI/pythia-14m/TRANSLATION/wmt09-fr-en/0-shot/logits/logits_part_0.pt"
logits = torch.load(logits_pth)
logits.shape

torch.Size([1, 256])

In [10]:
# import os
# import torch
# from collections import defaultdict
# import pandas as pd

# directory = "./models/experiment_6_logits_max_2"
# shape_distribution_per_model = defaultdict(lambda: defaultdict(int))

# # Walk through the directory
# for dirpath, _, filenames in os.walk(directory):
#     for filename in filenames:
#         if filename.endswith('.pt'):
#             # Get the parent directory of the file to use as the model name
#             model_name = os.path.basename(os.path.dirname(dirpath))
#             file_path = os.path.join(dirpath, filename)
#             tensor = torch.load(file_path).flatten()
#             tensor_shape = tensor.shape[0]  # Record the flattened size
#             shape_distribution_per_model[model_name][tensor_shape] += 1

# # Convert the distribution data into a list of tuples (model, tensor_size, count)
# data_for_plotting = []
# for model, shape_counts in shape_distribution_per_model.items():
#     for tensor_size, count in shape_counts.items():
#         data_for_plotting.append((model, tensor_size, count))

# # Convert the list into a DataFrame
# df = pd.DataFrame(data_for_plotting, columns=['Model', 'Tensor Size', 'Count'])

# # Now you can plot df using matplotlib or seaborn as shown previously

In [11]:
import glob

def load_logits(logits_pth):
    logit_dict = {}
    all_pths = sorted(glob.glob(os.path.join(logits_pth, "**/*.pt"), recursive=True))
    for pth in tqdm(all_pths):
        logits = torch.load(pth)
        id_ = int(pth.split("_")[-1].split(".")[0])
        logit_dict[id_] = logits
        # print(f"id: {id_}, logits shape: {logits.shape}")
    return logit_dict

model_logits_pth = "./models/experiment_6_logits_max_3/inference/EleutherAI/pythia-12b/TRANSLATION/wmt09-en-fr"
model_logits = load_logits(model_logits_pth)

  0%|          | 0/3027 [00:00<?, ?it/s]

100%|██████████| 3027/3027 [00:04<00:00, 654.08it/s] 


In [12]:
examples_common_pth = "./results/n-grams/exp_full/2/examples_dfs_0-shot_common_models.pkl"
# examples_common_pth = "./results/n-grams/wmt/pile/exp4/n_samples_None_fkeyFalse_rkeyFalse_fstopTrue_onlyalphaTrue/2/common/lang_dfs_filter_charsFalse_percentile0.999_n_gram2.pkl"
df = pickle.load(open(examples_common_pth, "rb"))
df_12b = df['pythia-12b']
df_fr_en = df_12b[df_12b['task'] == 'fr-en']
df_fr_en.head(1)

,lang_1,lang_2,value,example,coverage,task,query,sum,count,id,gold,src,ref,result,bleu
0,temps après,first time,186.0,"[temps après, first time]",0.066915,fr-en,"['temps après', 'first time']",186.0,1,2947_4,0,"Peu de temps après, le témoin, qui au début n'...","Shortly after, the witness, who, in a first ti...","A few minutes later, the witness, who at firs...",28.795572


In [13]:
from transformers import AutoTokenizer

model = 'EleutherAI/pythia-12b'
tokenizer = AutoTokenizer.from_pretrained(model)

def test_logits(model_logits, df_fr_en, tokenizer):
    for idx, row in df_fr_en.iterrows():
        id_ = int(row['id'].split("_")[0])  # row['id'] = id_task
        logits = model_logits[id_]
        gen_text = row['result']
        
        tokens = tokenizer(gen_text, return_tensors='pt', add_special_tokens=False)
        
        # Align logits with tokens, including special tokens
        # This assumes that logits are for each token in the sequence, including special tokens.
        aligned_logits = logits[:, :tokens['input_ids'].size(1)]
        
        # print(f"id: {id_}, gen: {gen_text}")
        print(f"logits: {logits.shape}, tokens: {tokens['input_ids'].shape}")
        print(f"Aligned logits shape: {aligned_logits.shape}")
        
        if idx > 5:
            break

test_logits(model_logits, df_fr_en, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


logits: torch.Size([1, 56]), tokens: torch.Size([1, 37])
Aligned logits shape: torch.Size([1, 37])
logits: torch.Size([1, 56]), tokens: torch.Size([1, 37])
Aligned logits shape: torch.Size([1, 37])
logits: torch.Size([1, 92]), tokens: torch.Size([1, 69])
Aligned logits shape: torch.Size([1, 69])
logits: torch.Size([1, 92]), tokens: torch.Size([1, 69])
Aligned logits shape: torch.Size([1, 69])
logits: torch.Size([1, 92]), tokens: torch.Size([1, 69])
Aligned logits shape: torch.Size([1, 69])
logits: torch.Size([1, 92]), tokens: torch.Size([1, 69])
Aligned logits shape: torch.Size([1, 69])
logits: torch.Size([1, 92]), tokens: torch.Size([1, 69])
Aligned logits shape: torch.Size([1, 69])


In [ ]:
def create_probs_column(model_logits, df, tokenizer):
    for idx, row in df_fr_en.iterrows():
        id_ = int(row['id'].split("_")[0])  # row['id'] = id_task
        
        probs = model_logits[id_].flatten()
        gen_text = row['result']
        
        tokens = tokenizer(gen_text, return_tensors='pt', add_special_tokens=False).flatten()
        len_tokens = len(tokens['input_ids'].size(1))
        
        # Align logits with tokens, including special tokens
        # This assumes that logits are for each token in the 
        # sequence, including special tokens.
        aligned_logits = logits[:len_tokens]
        probs = probs[:len_tokens]

        
        
        
        # print(f"id: {id_}, gen: {gen_text}")
        print(f"logits: {logits.shape}, tokens: {tokens['input_ids'].shape}")
        print(f"Aligned logits shape: {aligned_logits.shape}")
        
        if idx > 5:
            break

In [ ]:
df_fr_en['src'][0]

In [ ]:
ngram_counts = df_fr_en.groupby('lang_1')['value'].sum()
n_gram_distribution = ngram_counts / ngram_counts.sum()

In [ ]:
df_fr_en

In [ ]:
row_1 = df_fr_en['results']
ngram_combinations = wt.process_text(result_1, )

In [ ]:
def get_gen_ngrams_dist(df):
    def get_


In [ ]:
n_gram_distribution